In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_3982/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/17 05:00:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/17 05:00:06 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [30]:
import pandas as pd

# Sample data for the Candidates table
candidates_data = {
    'employee_id': [1, 9, 2, 11, 13, 4],
    'experience': ['Junior', 'Junior', 'Senior', 'Senior', 'Senior', 'Junior'],
    'salary': [10000, 10000, 20000, 20000, 50000, 40000]
}

# Convert data to DataFrame
candidates_df = pd.DataFrame(candidates_data)

# Display the DataFrame
print("Candidates table:")
print(candidates_df)

data = {
    'employee_id': [1, 9, 2, 11, 13, 4],
    'experience': ['Junior', 'Junior', 'Senior', 'Senior', 'Senior', 'Junior'],
    'salary': [10000, 10000, 80000, 80000, 80000, 40000]
}

# Create DataFrame
df = pd.DataFrame(data)



df_person = spark.createDataFrame(candidates_df)
df_person.createOrReplaceTempView("Candidates")

df_person = spark.createDataFrame(df)
df_person.createOrReplaceTempView("Candidates_1")



Candidates table:
   employee_id experience  salary
0            1     Junior   10000
1            9     Junior   10000
2            2     Senior   20000
3           11     Senior   20000
4           13     Senior   50000
5            4     Junior   40000


In [31]:
query = """
    with senior_salary as (
        select
            *,
            sum(salary) over(order by rk asc) as total_salary
        from (
            select
                *,
                row_number() over(order by salary asc) as rk
            from Candidates_1 c 
            where experience = 'Senior'
        )
    ),
    
    junior_salary as (
        select
            *,
            sum(salary) over(order by rk asc) as total_salary
        from (
            select
                *,
                row_number() over(order by salary asc) as rk
            from Candidates_1 c 
            where experience = 'Junior'
        )
    ),
    
    senior_get as (
        select
            *
        from senior_salary 
        where total_salary <= 70000
    )
    
    select 
        'Junior' as experience,
        count(*) as accepted_candidates
    from junior_salary 
    where total_salary <= (70000 - ifnull((select max(total_salary) from senior_get), 0))
    union all
    select
        'Senior' as experience,
        count(*) as accepted_candidates
    from senior_get
"""

In [32]:
spark.sql(query).show()

24/03/17 06:18:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 06:18:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 06:18:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 06:18:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 06:18:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 06:18:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 0

+----------+-------------------+
|experience|accepted_candidates|
+----------+-------------------+
|    Junior|                  3|
|    Senior|                  0|
+----------+-------------------+



In [8]:
query_1 = """
    select
        *
    from Candidates c 
    where experience = 'Senior'
"""

In [9]:
spark.sql(query_1).createOrReplaceTempView("SeniorSalary")
spark.sql(query_1).show()

+-----------+----------+------+
|employee_id|experience|salary|
+-----------+----------+------+
|          2|    Senior| 20000|
|         11|    Senior| 20000|
|         13|    Senior| 50000|
+-----------+----------+------+

